# Retrieval Augmented Generation

![](https://www.dailydoseofds.com/content/images/2024/10/rag.gif)
Source: [Daily Dose of DS](https://www.dailydoseofds.com/a-crash-course-on-building-rag-systems-part-1-with-implementations/)

Overview:
- Introduction to RAG with LlamaIndex
- Data ingestion
  - PDF
  - Web pages
  - Code
- Data splitting
  - Token splitting
  - Sentence splitting
  - Structured data splitting
  - Semantic chunking
- Vectorization
  - Embeddings
  - Vector storage
- Retrieval
  - Keyword search
  - Vector search
  - Hybrid search
- Advanced methods
  - Query rewriting
  - Multi-hop retrieval

In [ ]:
%load_ext rich
%load_ext autoreload
%autoreload 2


# Introduction to RAG with LlamaIndex

LlamaIndex is a library for working with large language models.
One of its main strengths is its ability to ingest documents into a vector index and use them to answer questions.
This is known as Retrieval Augmented Generation (RAG).

To start, we will use a low-code, high-level abstraction to build a basic PDF question-answering system.
We will read in PDFs, split them into chunks, embed them, and store them in a vector database.
Then, we will use an abstraction known as a `QueryEngine` that implements RAG to answer questions about the documents.

In [ ]:
# If we're in colab, use userdata to get the OPENAI_API_KEY
import os
from rich import print
from pathlib import Path

try:
    from google.colab import userdata
    print("Colab detected - setting up environment")
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    %pip install llama-index \
        llama-index-readers-web \
        thefuzz \
        gradio \
        chromadb \
        llama-index-embeddings-huggingface \
        llama-index-vector-stores-chroma \
        llama-index-retrievers-bm25 \
        llama-index-llms-gemini \
        docling \
        llama-index-readers-docling \
        llama-index-node-parser-docling \
        PyStemmer
        
except:
    print("Not in colab - using local environment variables.")
    from dotenv import load_dotenv
    load_dotenv("../.env")


In [ ]:
import os
import requests

# Create data directory if it doesn't exist
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Download the PDF file
pdf_url = "https://arxiv.org/pdf/2407.21783"
pdf_path = os.path.join(data_dir, "2407.21783.pdf")

if not os.path.exists(pdf_path):
    response = requests.get(pdf_url)
    with open(pdf_path, "wb") as f:
        f.write(response.content)
    print(f"Downloaded PDF to {pdf_path}")
else:
    print(f"PDF already exists at {pdf_path}")


The first thing we need to do is to load the data.
For general documents like PDFs, LlamaIndex provides a nice abstraction known as a `SimpleDirectoryReader` that can load data from a directory.
In the cell below, we use it to load the data from the `data` directory.

In [6]:
from llama_index.core.readers import SimpleDirectoryReader
documents = SimpleDirectoryReader(data_dir).load_data()

Let's take a look at the first document.
Most prominently, we get the text of the document.
By default, we also get a lot of useful information, like the page number file name, file path, type, size, etc.
When we load lots of documents, this type of information becomes important to keep track of.

In [ ]:
print(documents[0])

Now that we've loaded the data, we need to vectorize it.
We will use a combination of an embedding model and a vector database to store the vectors.
In the cell below, we use a HuggingFace embedding model to embed the documents.
We also use torch to determine the device to use for the embedding model (`mps` for Mac GPUs, `cuda` for Nvidia GPUs, and `cpu` otherwise).

In [8]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from torch.backends.mps import is_available as is_mps_available
from torch.cuda import is_available as is_cuda_available

if is_mps_available():
    device = "mps"
elif is_cuda_available():
    device = "cuda"
else:
    device = "cpu"

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-large-en-v1.5", device=device)
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

Finally, let's set up our RAG query engine.
If we want to perform simple question-answering, we can use the `as_query_engine` method.
If we want to perform chat with history, we can use the `as_chat_engine` method.
We can see both below 👇.


In [9]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
llm = OpenAI(model="gpt-4o-mini")
Settings.llm = llm # set gpt-4o-mini as the default llm

query_engine = index.as_query_engine(llm=llm)
chat_engine = index.as_chat_engine(llm=llm)

Let's see how the query enine works.
We start by passing it a question, then it uses the retriever to find the most relevant documents.
Finally, it uses the LLM to answer the question.

In [10]:
response = query_engine.query("How many new Llama models models are mentioned in the paper?")

Below we see the response text.
But there's also some additional information that we can access, including the source nodes.
These are the nodes that the retriever used to answer the question.
We can see that the retriever found several nodes that are relevant to the question, then the LLM used at least one of them to answer the question.

In [ ]:
print(response.response)

Taking a look at the first source node, we can see that it is a node that contains part of the document that is relevant to the question.
It also contains the `score`, which is the similarity between the question and the node.


In [ ]:
print(response.source_nodes[0])

Now let's see if our chat enigine comes up with the same answer.

In [ ]:
chat_response = chat_engine.chat("How many new Llama models models are mentioned in the paper?")
print(chat_response.response)
print(chat_response.source_nodes[0])

# Data ingestion

Data often comes in many different formats.
It may come in the form of a PDF, a web page, a code file, etc.
We may need some specific processing pipelines to extract the text from these documents, split them correctly, and vectorize them.

Luckily, LlamaIndex (and other libraries) provide lots of built-in and add-on tools to help you ingest almost any data type.
Instead of loading a PDF, let's load a web page instead.
We will use one of the classes provided by [`llama-index-readers-web`](https://llamahub.ai/l/readers/llama-index-readers-web?from=readers) to load data from a web page.

In this section, we will:
- Load a web page as Markdown
- Split it into chunks following the structured format of the Markdown
- Embed the chunks
- Store the chunks in a vector database
- Create a query engine from the vector database and use it to answer a question


In [14]:
from llama_index.readers.web import SimpleWebPageReader
from llama_index.core.node_parser import MarkdownNodeParser
from llama_index.core.ingestion import IngestionPipeline
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb


In [15]:
web_docs = SimpleWebPageReader(html_to_text=True).load_data(['https://en.wikipedia.org/wiki/Wikipedia'])

In [16]:
collection = chromadb.EphemeralClient().create_collection("wikipedia", get_or_create=True)
vector_store = ChromaVectorStore(collection)

In [17]:
pipeline = IngestionPipeline(
    transformations=[
        MarkdownNodeParser.from_defaults(),
        embed_model,
    ], 
    vector_store=vector_store
)

In [18]:
nodes = pipeline.run(documents=web_docs)

In [19]:
index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)

In [ ]:
llm = OpenAI(model="gpt-4o-mini")
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("How many languages there exactly? Quote the exact text as well.")
print(response.response)

In [ ]:
# Use fuzzywuzzy to find the closest match in the source text
from thefuzz import fuzz, process
# Get the top matching line of text from the source_text_quote
top_match, match_score = process.extractOne(response.response, response.source_nodes[0].text.splitlines(), scorer=fuzz.ratio)
assert top_match in response.source_nodes[0].text
print(f"Quote from source: '{top_match}'")

## OCR

There are times where the PDF is not in a format that can be easily read into text.
In these cases, we will need to use optical character recognition (OCR) to convert the images to text.
There are many libraries and cloud services that can do this, but for this example, we will use the `docling` library since our example document is a PDF.
We will then ingest the text into LlamaIndex and use it to answer a question.

![](https://ds4sd.github.io/docling/assets/docling_processing.png)

In [28]:
from llama_index.readers.docling import DoclingReader
from llama_index.node_parser.docling import DoclingNodeParser
from llama_index.core.schema import Document
from llama_index.core import StorageContext

In [23]:
reader = DoclingReader(export_type=DoclingReader.ExportType.JSON)

In [ ]:
docling_docs = reader.load_data(pdf_path)

In [ ]:
len(docling_docs)

In [42]:
# We need to set the text template to "{content}" because the default is "{metadata}\n\n{content}",
# and LlamaIndex will try to embed the metadata as well. The metadata is not useful at serach time.
docling_docs[0].text_template
docling_docs[0].text_template = "{content}"

In [ ]:
docling_node_parser = DoclingNodeParser()
docling_nodes = docling_node_parser.get_nodes_from_documents(docling_docs)
len(docling_nodes)

In [ ]:
docling_nodes[0].metadata

In [46]:
index = VectorStoreIndex(nodes=docling_nodes, embed_model=embed_model)

In [ ]:
query_engine = index.as_query_engine(llm=llm)
response = query_engine.query("How many new Llama models are mentioned in the paper?")
print(response.response)

In [ ]:
print(response.source_nodes[1])

# Data splitting

Many times, it's impractical to embed the entire document, and expensive to feed the entire document to the LLM.
Instead, we can split the document into chunks, embed the chunks, and use a retrieval method to find the most relevant chunks.
There are naïve methods that split texts into chunks of a specific length with some overlap;
there are methods that use the structure of the document to split it into sections (e.g. sections, figures, tables);
and there are more advanced methods that use semantic similarity to group the text into chunks.

Since the OCR'd text is just one very long Markdown string, we need to split it into chunks.
One nice way to do that is use the inherent structure of Markdown to split it into sections.
We do that here with LlamaIndex's `MarkdownNodeParser`.

In [ ]:
docling_md_docs = DoclingReader(export_type=DoclingReader.ExportType.MARKDOWN).load_data(pdf_path)


In [71]:
from llama_index.core.node_parser import MarkdownNodeParser
from IPython.display import Markdown, display

In [68]:
md_nodes = MarkdownNodeParser.from_defaults().get_nodes_from_documents(documents=docling_md_docs)

In [ ]:
len(md_nodes)

In [ ]:
display(Markdown(md_nodes[9].text))


In [ ]:
md_index = VectorStoreIndex(nodes=md_nodes, embed_model=embed_model)
md_query_engine = md_index.as_query_engine(llm=llm)
response = md_query_engine.query("How many llama3 models are there?")
print(response.response)
print(response.source_nodes[0].text)

---

# Retrieval

## Starting simple: bm25

BM25 is a simple retrieval method that uses the BM25 algorithm to score the relevance of each document to the query.
The BM25 algorithm is a probabilistic retrieval model that uses the term frequency and inverse document frequency of the query terms to score the relevance of each document.
You should be familiar with the basic idea of tf-idf from your NLP class - you can think of BM25 as a generalization of tf-idf that takes into account more factors.

In [82]:
from llama_index.retrievers.bm25 import BM25Retriever

In [86]:
bm25 = BM25Retriever.from_defaults(nodes=md_nodes)

In [ ]:
for node in bm25.retrieve("How many llama3 models are there?"):
    print(f"Score: {node.score:.4f}\nText:\n{node.node.text[:500]}...")

## Dense retrieval (vector search)

BM25 is a simple and fast method that depends on word matching.
But if we want to do more complex retrieval, we can use dense retrieval.
We represent both our query and documents as vectors and use a similarity metric to find the most relevant documents.
This is what's been going on under the hood in the previous examples using `VectorStoreIndex`.

Since most of the mechanics are taken care for us under the hood, let's examine what goes on under the hood.

In [88]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt


In [89]:
sample_documents = [
    "My favorite type of dog is a golden retriever.",
    "I like to eat pizza with my friends.",
    "I like to go to the gym in the morning.",
    "I like to play basketball with my friends.",
]

embeddings = np.array(embed_model.get_text_embedding_batch(sample_documents))

query = "What do I like to do with friends?"
query_embedding = np.array(embed_model.get_text_embedding(query))

In [ ]:
cosine_similarity(query_embedding.reshape(1, -1), embeddings).squeeze()

Once we have a `VectorStoreIndex`, we can use the `as_retriever` method to get a retriever object.
This uses dense retrieval under the hood, since it already has an embedding model as a part of the class.
Here, we use the `similarity_top_k` parameter to limit the number of results to 2 and show the cosine similarity score along with the beginning of the text.

In [ ]:
query = "How many llama3 models are there?"
top_results = md_index.as_retriever(similarity_top_k=2).retrieve(query)
for result in top_results:
    print(f"Score: {result.score:.4f}\nText:\n{result.node.text[:500]}...")

## Hybrid search: query rewriting and reciprocal ranking

Sometimes, you may want several methods of searching over your data, then combining the results.
This is known as hybrid search.

Haveing multiple retrievers may not mean having separate objects - we may just have multiple queries.
In this example, we'll use an LLM to rewrite our query into multiple queries, then use a dense retriever to find the most relevant documents.
Finally, we'll use reciprocal ranking to re-rank the results.


In [98]:
from llama_index.core.retrievers import QueryFusionRetriever

In [101]:
dense_retriever = md_index.as_retriever(similarity_top_k=5)
hybrid_retriever = QueryFusionRetriever(
    [dense_retriever],
    num_queries=3,
    use_async=False,
    mode='reciprocal_rerank',
    verbose=True
)

In [ ]:
results = hybrid_retriever.retrieve("How many llama3 models are there?")
for result in results:
    print(f"Score: {result.score:.4f}\nText:\n{result.node.text[:500]}...")

# Reranking

Often, retrieval only gives us a good first pass at finding relevant documents.
We can use re-ranking to improve the results.
There are several re-ranking methods, but in this case we'll use a cross-encoder to re-rank the results.
We'll also introduce a more low-level method for organizing LlamaIndex flows called workflows.

In [130]:
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.schema import NodeWithScore
from llama_index.core.llms import ChatMessage
from llama_index.core.workflow import (
    Workflow,
    Context,
    step,
    StartEvent,
    StopEvent,
    Event
)
from typing import List


Let's walk throught the code below step by step.
First, we define the events that will be passed between the steps.
This includes a retrieval result, a re-ranking result, and a prompt for the LLM.
The start and stop are already taken care of for us by `StartEvent` and `StopEvent`.

Next, we define our `RAGWithReRank` workflow.
We initialize it with an index and LLM, which we will use for retrieval and answering and the cross-encoder for re-ranking.
We also define exactly what to do for each step of the workflow.
It knows what to do because of the `@step` decorator and the types annotations of each method.

Finally, we can run the workflow with a query.
The workflow will automatically handle passing the events between the steps, so we don't have to worry about that.
We can observe that the results are just what we expect as we saw from the previous examples.
One advantage of this method is that it's much more flexible - you can include arbitrary code, loops, conditionals, etc.

For more detailed documentation on workflows, see [here](https://docs.llamaindex.ai/en/stable/understanding/workflows/).

In [155]:
# Define the events that will be passed between the steps
class RetrievalResult(Event):
    results: List[NodeWithScore]

class ReRankResult(Event):
    results: List[NodeWithScore]

class RagPrompt(Event):
    prompt: str

# Define the workflow
class RAGWithReRank(Workflow):
    # Initialize the workflow with an index and LLM
    def __init__(self, index: VectorStoreIndex = md_index, llm: OpenAI = OpenAI(model="gpt-4o-mini")):
        super().__init__(timeout=10, verbose=False)
        self.index = index
        self.llm = llm
        self.reranker = SentenceTransformerRerank(similarity_top_k=5)

    # The start method is called when the workflow is run.
    # This is the first step of the workflow that takes in a query and returns a retrieval result.
    # Since we want to use a course retriever, we use a top k of 20.
    @step
    async def start(self, ctx: Context, ev: StartEvent) -> RetrievalResult:
        query = ev.query
        await ctx.set('query', query)
        results = self.index.as_retriever(similarity_top_k=20).retrieve(query)
        return RetrievalResult(results=results)
    
    # This step takes the retrieval results and re-ranks them.
    # Notice that the similarity top k is set to 5, so we only keep the top 5 results.
    # This is smaller than the retrieval step since this is about refining a smaller number of results.
    @step
    async def rerank(self, ctx: Context, ev: RetrievalResult) -> ReRankResult:
        results = ev.results
        query = await ctx.get('query')
        reranked = self.reranker.postprocess_nodes(nodes=results, query_str=query)
        return ReRankResult(results=reranked)
    
    # This step creates a prompt for the LLM.
    # It does this by joining the re-ranked results into a single string and formatting it with the query.
    @step
    async def create_prompt(self, ctx: Context, ev: ReRankResult) -> RagPrompt:
        reranked_results = ev.results
        await ctx.set('reranked_results', reranked_results)
        query = await ctx.get('query')
        reranked_str = '\n\n'.join(i.text for i in reranked_results)
        prompt = f"""\
Here is some relevant context:
--------------------------------
{reranked_str}
--------------------------------
Based on the above information and not prior knowledge, please answer the question.
Question:
{query}
"""
        return RagPrompt(prompt=prompt)

    # This step takes the prompt and uses the LLM to answer the question.
    # It also takes the re-ranked results and attaches them to the response in case we want to see the nodes.
    @step
    async def answer(self, ctx: Context, ev: RagPrompt) -> StopEvent:
        prompt = ev.prompt
        ranked_results = await ctx.get('reranked_results')
        messages = [
            ChatMessage.from_str(prompt)
        ]
        answer = await self.llm.achat(messages)
        result = {
            "response": answer,
            "source_nodes": ranked_results
        }
        return StopEvent(result=result)


In [156]:
rag = RAGWithReRank()

In [ ]:
await rag.run(query="How many llama3 models are there?")

In [1]:
from llama_index.utils.workflow import draw_all_possible_flows

In [ ]:
from llama_index.utils.workflow import draw_all_possible_flows
from IPython.display import HTML

draw_all_possible_flows(MyWorkflow, filename="flow.html")

with open("flow.html", "r") as f:
    display(HTML(f.read()))


# Exercise: Chat with PDF (30 minutes)

Your goal in this exercise is to create a Gradio interface for a question-answering system.
Your application should:
- Use the query engine created above to answer questions about the uploaded PDF
- Display the question and answer in the UI. If using QueryEngine, use a question and answer format. If using ChatEngine, use a chat format.

If you need a challenge:
- Use the `gr.File` component to allow the user to upload ANY pdf and ask question about it.


In [104]:
import gradio as gr
from tempfile import TemporaryDirectory

In [ ]:
# In the ingest_documents function:
def ingest_documents(file_obj):
    """
    Process an uploaded PDF file and create a query engine for it.
    
    Args:
        file_obj: Gradio file upload object containing the PDF
        
    Returns:
        tuple: (chat_engine, cleared_file_upload, filename)
    """
    if file_obj is None:
        return None, None, ""
        
    with TemporaryDirectory() as temp_dir:
        file_path = os.path.join(temp_dir, 'tmp.pdf')
        # Get the file bytes from the Gradio upload object
        file_bytes = open(file_obj, "rb").read()
        
        with open(file_path, "wb") as f:
            f.write(file_bytes)
            
        documents = SimpleDirectoryReader(temp_dir).load_data()
        llm = OpenAI(model="gpt-4o-mini")
        index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)
        chat_engine = index.as_chat_engine(llm=llm)
        
    return chat_engine, None, Path(file_obj.name).name, []

def chat(message, history, chat_engine):
    history.append(
        {
            "role": "user",
            "content": message
        }
    )
    response = chat_engine.chat(message)
    history.append({
        "role": "assistant",
        "content": response.response
    })
    return history, None

# In the Blocks interface:
with gr.Blocks() as demo:
    chat_engine_state = gr.State()  # Renamed for clarity
    gr.Markdown("## RAG Demo: Question answering with a PDF")
    with gr.Row():
        with gr.Column(scale=1):   
            file_upload = gr.File(label="Upload a PDF file", file_types=[".pdf"], file_count="single")
            submit_button = gr.Button("Submit")
            pdf_name = gr.Textbox(label="You are asking about...")
        with gr.Column(scale=5):
            chatbot = gr.Chatbot(label="Chat with the uploaded PDF", type='messages')
            input = gr.Textbox(label="Enter a question", interactive=True)

    # Update to only store the chat_engine in the State
    submit_button.click(
        fn=ingest_documents, 
        inputs=file_upload, 
        outputs=[chat_engine_state, file_upload, pdf_name, chatbot]
    )
    input.submit(
        fn=chat, 
        inputs=[input, chatbot, chat_engine_state], 
        outputs=[chatbot, input]
    )

demo.launch(debug=False)